In [43]:
#imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
from IPython.display import display
from sklearn.preprocessing import MinMaxScaler
from plotly.subplots import make_subplots

In [44]:

pio.templates.default = "plotly_white"

#cores/classificações
ordem_classes = ['Excelente', 'Boa', 'Ok', 'Ruim', 'Péssima']
paleta_cores = {
    'Excelente': '#7b68ee',
    'Boa': '#65c99f', 
    'Ok': '#3f3f3f',
    'Ruim': '#01abdf',
    'Péssima': "#a0a0a0"
}

In [45]:

#carregamento e limpeza dos dados
print("Iniciando carregamento e limpeza dos dados...")
dataset = pd.read_csv('Dados 1.csv', encoding='utf-8-sig')
dataset.replace(["NA", "NM"], np.nan, inplace=True)

cols_to_exclude = ['ENTITY_NAME', 'TICKER', 'INDUSTRY', 'SECTOR', 'PRIMARY_INDUSTRY']
numeric_cols = dataset.columns.drop(cols_to_exclude)
dataset[numeric_cols] = dataset[numeric_cols].apply(
    lambda x: pd.to_numeric(x.astype(str).str.replace(',', ''), errors='coerce')
)
colunas_retorno = ['ROE_LTM', 'ROA_LTM', 'ROC_LTM']

#preenchimento por setor
colunas_chave = [
    'ROE_LTM', 'DEBT_TO_EBITDA_LTM', 'EV/EBITDA_LTM', 'NET_MARGIN_LTM', 
    'PE_LTM', 'PBV_LTM', 'INTEREST_COVERAGE_LTM', 'LFCF_MARGIN_LTM',
    'ROA_LTM', 'CURRENT_RATIO_LTM', 'ASSET_TURNOVER_LTM', 'MARKETCAP'
]
colunas_existentes = [col for col in colunas_chave if col in dataset.columns]
dataset[colunas_existentes] = dataset.groupby('SECTOR')[colunas_existentes].transform(
    lambda x: x.fillna(x.median())
)

original_rows = len(dataset)
dataset.dropna(subset=colunas_existentes, inplace=True)
print(f"Limpeza concluída: {original_rows - len(dataset)} linhas removidas")
print(f"Shape final: {dataset.shape}")

Iniciando carregamento e limpeza dos dados...
Limpeza concluída: 0 linhas removidas
Shape final: (408, 197)


In [46]:
# ============= SEÇÃO 3: CÁLCULO DE MÉTRICAS =============
def calculate_cagr(start, end, periods=3):
    if pd.isna(start) or pd.isna(end) or start <= 0 or end <= 0:
        return np.nan
    return (end / start) ** (1 / periods) - 1

print("\nCalculando métricas de crescimento...")
dataset['Revenue_CAGR'] = dataset.apply(lambda row: calculate_cagr(row['TOTAL_REV_FY2022'], row['TOTAL_REV_LTM']), axis=1)
dataset['EBITDA_CAGR'] = dataset.apply(lambda row: calculate_cagr(row['EBITDA_FY2022'], row['EBITDA_LTM']), axis=1)
dataset['Net_Income_CAGR'] = dataset.apply(lambda row: calculate_cagr(row['NET_INCOME_FY2022'], row['NET_INCOME_LTM']), axis=1)

dataset[['Revenue_CAGR', 'EBITDA_CAGR', 'Net_Income_CAGR']] = dataset[['Revenue_CAGR', 'EBITDA_CAGR', 'Net_Income_CAGR']].fillna(0)

dataset['INTEREST_COVERAGE_LTM'] = dataset['EBIT_LTM'] / dataset['INTEREST_EXPENSE_LTM'].replace(0, np.nan)
dataset['INTEREST_COVERAGE_LTM'].fillna(0, inplace=True)

dataset['ASSET_TURNOVER_LTM'] = dataset['TOTAL_REV_LTM'] / dataset['TOTAL_ASSETS_LTM'].replace(0, np.nan)
dataset['ASSET_TURNOVER_LTM'].fillna(0, inplace=True)


Calculando métricas de crescimento...


C:\Users\Augusto\AppData\Local\Temp\ipykernel_23312\1390327926.py:15: FutureWarning:

A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.



C:\Users\Augusto\AppData\Local\Temp\ipykernel_23312\1390327926.py:18: FutureWarning:

A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, in

In [47]:
#percentis por setor e classificação
sector_thresholds = {}
metrics_to_percentile = {
    'EBITDA_MARGIN_LTM': False, 'NET_MARGIN_LTM': False, 'EV/EBITDA_LTM': True,
    'PE_LTM': True, 'PBV_LTM': True, 'DEBT_TO_EBITDA_LTM': True,
    'INTEREST_COVERAGE_LTM': False, 'LFCF_MARGIN_LTM': False, 'ROE_LTM': False,
    'ROA_LTM': False, 'CURRENT_RATIO_LTM': False, 'ASSET_TURNOVER_LTM': False
}
grouped_by_sector = dataset.groupby('SECTOR')
for metric in metrics_to_percentile.keys():
    quantiles = grouped_by_sector[metric].quantile([0.2, 0.4, 0.6, 0.8]).unstack()
    sector_thresholds[metric] = quantiles.rename(columns={0.2: 'p20', 0.4: 'p40', 0.6: 'p60', 0.8: 'p80'})

def assign_points(class_val):
    mapping = {'Excelente': 2, 'Boa': 1, 'Ok': 0, 'Ruim': -1, 'Péssima': -2}
    return mapping.get(class_val, 0)

def classify_metric(value, sector, metric_name, invert, thresholds):
    if pd.isna(value):
        return 'Ok'
    
    th_p20, th_p40, th_p60, th_p80 = thresholds
    try:
        sector_th = sector_thresholds[metric_name].loc[sector]
        th_p20, th_p40, th_p60, th_p80 = sector_th['p20'], sector_th['p40'], sector_th['p60'], sector_th['p80']
    except (KeyError, ValueError):
        pass
    
    if invert:
        if value <= th_p20: return 'Excelente'
        elif value <= th_p40: return 'Boa'
        elif value <= th_p60: return 'Ok'
        elif value <= th_p80: return 'Ruim'
        else: return 'Péssima'
    else:
        if value >= th_p80: return 'Excelente'
        elif value >= th_p60: return 'Boa'
        elif value >= th_p40: return 'Ok'
        elif value >= th_p20: return 'Ruim'
        else: return 'Péssima'

def assign_class_cagr(row):
    rev, ebit, net = row['Revenue_CAGR'], row['EBITDA_CAGR'], row['Net_Income_CAGR']
    if [rev > 0.15, ebit > 0.15, net > 0.15].count(True) >= 2: return 'Excelente'
    elif [rev > 0.05, ebit > 0.05, net > 0.05].count(True) >= 2: return 'Boa'
    elif [rev >= 0, ebit >= 0, net >= 0].count(True) >= 2: return 'Ok'
    elif [rev < 0, ebit < 0, net < 0].count(True) >= 2: return 'Ruim'
    else: return 'Péssima'

def assign_class_from_metrics(row, metric_configs, score_thresholds):
    scores = [assign_points(classify_metric(row[m], row['SECTOR'], m, inv, th)) 
              for m, inv, th in metric_configs]
    total = sum(scores)
    
    if total >= score_thresholds[0]: return 'Excelente'
    elif total >= score_thresholds[1]: return 'Boa'
    elif total >= score_thresholds[2]: return 'Ok'
    elif total >= score_thresholds[3]: return 'Ruim'
    else: return 'Péssima'

# ============= NOVA FUNÇÃO: MÚLTIPLOS POR MEDIANA SETORIAL =============
def classify_multiplos_setorial(row):
    sector = row['SECTOR']
    score = 0
    
    multiplos = ['EV/EBITDA_LTM', 'PE_LTM', 'PBV_LTM']
    
    for mult in multiplos:
        value = row[mult]
        if pd.isna(value):
            continue
        
        try:
            sector_median = dataset[dataset['SECTOR'] == sector][mult].median()
            
            if pd.isna(sector_median):
                continue
                
            # Quanto menor o múltiplo em relação à mediana, melhor
            ratio = value / sector_median
            
            if ratio <= 0.7:  # 30% abaixo da mediana
                score += 2
            elif ratio <= 0.85:  # 15% abaixo da mediana
                score += 1
            elif ratio <= 1.15:  # Próximo da mediana (±15%)
                score += 0
            elif ratio <= 1.30:  # 30% acima da mediana
                score -= 1
            else:  # Muito acima da mediana
                score -= 2
        except:
            continue
    
    # Classificação baseada no score total
    if score >= 4: return 'Excelente'
    elif score >= 2: return 'Boa'
    elif score >= -1: return 'Ok'
    elif score >= -3: return 'Ruim'
    else: return 'Péssima'

# ============= NOVA FUNÇÃO: DIVIDENDOS COM HISTÓRICO =============
def assign_class_dividends_historico(row):
    # Coletar histórico de dividendos e payout
    div_years = []
    payout_years = []
    
    for year in ['LTM', 'FY2024', 'FY2023', 'FY2022', 'FY2021']:
        div_col = f'COMMON_DIV_DECLARED_{year}'
        payout_col = f'PAYOUT_RATIO_{year}'
        
        if div_col in row.index and not pd.isna(row[div_col]) and row[div_col] > 0:
            div_years.append(row[div_col])
        
        if payout_col in row.index and not pd.isna(row[payout_col]):
            payout_years.append(row[payout_col])
    
    # Se não tem histórico de dividendos
    if len(div_years) == 0:
        return 'Péssima'
    
    # Calcular dividend yield médio histórico
    price = row['PRICE_CLOSE_2025']
    if pd.isna(price) or price <= 0:
        return 'Péssima'
    
    avg_div = np.mean(div_years)
    avg_yield = avg_div / price
    
    # Calcular payout médio histórico
    avg_payout = np.mean(payout_years) if len(payout_years) > 0 else None
    
    # Verificar consistência (pagou dividendos em pelo menos 3 dos últimos 5 anos)
    consistency = len(div_years)
    
    # Verificar crescimento de dividendos
    growth = 0
    if len(div_years) >= 3:
        recent = np.mean(div_years[:2])  # Últimos 2 anos
        older = np.mean(div_years[-2:])   # Anos mais antigos
        if older > 0:
            growth = (recent - older) / older
    
    # Pontuação composta
    score = 0
    
    # Yield médio
    if avg_yield >= 0.05:
        score += 2
    elif avg_yield >= 0.03:
        score += 1
    elif avg_yield >= 0.015:
        score += 0
    else:
        score -= 1
    
    # Consistência
    if consistency >= 4:
        score += 2
    elif consistency >= 3:
        score += 1
    elif consistency >= 2:
        score += 0
    else:
        score -= 1
    
    # Payout sustentável
    if avg_payout is not None:
        if avg_payout <= 60:
            score += 1
        elif avg_payout <= 80:
            score += 0
        else:
            score -= 1
    
    # Crescimento
    if growth > 0.10:
        score += 1
    elif growth > 0:
        score += 0
    else:
        score -= 1
    
    # Classificação final
    if score >= 5: return 'Excelente'
    elif score >= 3: return 'Boa'
    elif score >= 0: return 'Ok'
    elif score >= -2: return 'Ruim'
    else: return 'Péssima'

#aplicando classificações
print("Aplicando classificações...")
dataset['class_CAGR'] = dataset.apply(assign_class_cagr, axis=1)

margins_metrics = [('EBITDA_MARGIN_LTM', False, [0.05, 0.10, 0.15, 0.25]),
                   ('NET_MARGIN_LTM', False, [0.00, 0.05, 0.10, 0.15])]
dataset['class_Margins'] = dataset.apply(lambda r: assign_class_from_metrics(r, margins_metrics, [3, 2, 0, -1]), axis=1)

# ALTERADO: Múltiplos agora usa mediana setorial
dataset['class_Multiplos'] = dataset.apply(classify_multiplos_setorial, axis=1)

qualidade_metrics = [('DEBT_TO_EBITDA_LTM', True, [1.5, 2.5, 3.5, 5.0]),
                     ('INTEREST_COVERAGE_LTM', False, [1.5, 3.0, 5.0, 8.0]),
                     ('LFCF_MARGIN_LTM', False, [0.00, 0.03, 0.07, 0.12])]
dataset['class_Qualidade'] = dataset.apply(lambda r: assign_class_from_metrics(r, qualidade_metrics, [4, 2, 0, -2]), axis=1)

profitability_metrics = [('ROE_LTM', False, [0.05, 0.10, 0.15, 0.20]),
                         ('ROA_LTM', False, [0.01, 0.03, 0.06, 0.10]),
                         ('CURRENT_RATIO_LTM', False, [1.0, 1.3, 1.8, 2.5]),
                         ('ASSET_TURNOVER_LTM', False, [0.4, 0.6, 0.9, 1.2])]
dataset['class_Profitability'] = dataset.apply(lambda r: assign_class_from_metrics(r, profitability_metrics, [5, 2, 0, -3]), axis=1)

print("Calculando classificação de Dividendos...")
# ALTERADO: Dividendos agora considera histórico completo
dataset['class_Dividends'] = dataset.apply(assign_class_dividends_historico, axis=1)

# ============= SCORE COMPOSTO COM DIVIDENDOS =============
pilares = ['class_CAGR', 'class_Margins', 'class_Multiplos', 'class_Qualidade', 
           'class_Profitability', 'class_Dividends']
for pilar in pilares:
    dataset[f'{pilar}_score'] = dataset[pilar].apply(assign_points)

# Composite score ponderado
dataset['composite_score'] = (
    dataset['class_CAGR_score'] * 1.0 +
    dataset['class_Margins_score'] * 1.0 +
    dataset['class_Multiplos_score'] * 1.0 +
    dataset['class_Qualidade_score'] * 1.0 +
    dataset['class_Profitability_score'] * 1.0 +
    dataset['class_Dividends_score'] * 1.0
)

def assign_class_composite(score):
    if score >= 8: return 'Excelente'
    elif score >= 4: return 'Boa'
    elif score >= -2: return 'Ok'
    elif score >= -7: return 'Ruim'
    else: return 'Péssima'

dataset['class_Composto'] = dataset['composite_score'].apply(assign_class_composite)

print(f"Classificações concluídas!")
print(f"Distribuição Múltiplos: {dataset['class_Multiplos'].value_counts().to_dict()}")
print(f"Distribuição Dividendos: {dataset['class_Dividends'].value_counts().to_dict()}")

Aplicando classificações...
Calculando classificação de Dividendos...
Classificações concluídas!
Distribuição Múltiplos: {'Ok': 184, 'Ruim': 65, 'Péssima': 56, 'Boa': 55, 'Excelente': 48}
Distribuição Dividendos: {'Péssima': 165, 'Boa': 88, 'Ok': 78, 'Excelente': 45, 'Ruim': 32}


In [48]:
# ============= PARTE 4: VISUALIZAÇÕES =============

# Gráfico 1: Dashboard com 7 pilares (3x3)
pilares_completo = ['class_CAGR', 'class_Margins', 'class_Multiplos', 'class_Qualidade', 
                    'class_Profitability', 'class_Dividends', 'class_Composto']

fig_pilares = make_subplots(
    rows=3, cols=3,
    subplot_titles=[p.replace('class_', '').replace('_', ' ') for p in pilares_completo],
    vertical_spacing=0.15,
    horizontal_spacing=0.10
)

for idx, pilar in enumerate(pilares_completo):
    row = (idx // 3) + 1
    col = (idx % 3) + 1
    
    counts = dataset[pilar].value_counts().reindex(ordem_classes, fill_value=0)
    
    fig_pilares.add_trace(
        go.Bar(
            x=counts.index,
            y=counts.values,
            text=[f'{v}' for v in counts.values],
            textposition='outside',
            marker_color=[paleta_cores[c] for c in counts.index],
            showlegend=False,
            hovertemplate='<b>%{x}</b><br>Empresas: %{y}<extra></extra>'
        ),
        row=row, col=col
    )

fig_pilares.update_xaxes(tickangle=-45, tickfont=dict(size=9))
fig_pilares.update_yaxes(title_text='Quantidade')
fig_pilares.update_layout(
    height=1100,
    title_text='Dashboard de Classificações por Pilar (com Dividendos)',
    title_x=0.5,
    title_font_size=18,
    showlegend=False
)

fig_pilares.show()

# Gráfico 2: Heatmap Top 30 (com dividendos)
print("\n🏆 Top 30 Empresas - Heatmap:")

df_melhores = dataset[dataset['class_Composto'].isin(['Excelente', 'Boa'])].copy()
ranking = df_melhores.sort_values('composite_score', ascending=False).head(30)

colunas_ranking = ['ENTITY_NAME', 'TICKER', 'SECTOR', 'composite_score', 'class_Composto',
                   'class_CAGR', 'class_Margins', 'class_Multiplos', 
                   'class_Qualidade', 'class_Profitability', 'class_Dividends']

display(ranking[colunas_ranking].style.background_gradient(cmap='Purples', subset=['composite_score']))

# Heatmap dos pilares
class_map = {'Excelente': 2, 'Boa': 1, 'Ok': 0, 'Ruim': -1, 'Péssima': -2}
heatmap_data = ranking[colunas_ranking[5:]].applymap(lambda x: class_map.get(x, 0))

fig_heatmap_top = go.Figure(data=go.Heatmap(
    z=heatmap_data.values,
    x=['CAGR', 'Margins', 'Múltiplos', 'Qualidade', 'Rentabilidade', 'Dividendos'],
    y=ranking['ENTITY_NAME'].values,
    colorscale=[[0, '#ff765e'], [0.25, "#c9df01"], [0.5, "#ffffe0"], [0.75, "#009e5c"], [1, "#00641E"]],
    text=heatmap_data.values,
    texttemplate='%{text}',
    textfont={"size": 9},
    colorbar=dict(title="Score", tickvals=[-2, -1, 0, 1, 2], 
                  ticktext=['Péssima', 'Ruim', 'Ok', 'Boa', 'Excelente'])
))

fig_heatmap_top.update_layout(
    title='Pontos Fortes e Fracos das Top 30 Empresas (com Dividendos)',
    height=900,
    yaxis_title='Empresa',
    xaxis_title='Pilar',
    title_font_size=16
)

fig_heatmap_top.show()

# Gráfico 3: Análise Setorial (ROE e Dívida)
print("\n🏭 Análise Setorial:")

sector_roe = dataset.groupby('SECTOR')['ROE_LTM'].median().sort_values(ascending=False)
sector_debt = dataset.groupby('SECTOR')['DEBT_TO_EBITDA_LTM'].median().sort_values(ascending=True)

fig_setores = make_subplots(
    rows=1, cols=2,
    subplot_titles=('ROE Mediano', 'Setores - Menor Endividamento'),
    horizontal_spacing=0.15
)

fig_setores.add_trace(
    go.Bar(
        y=sector_roe.index,
        x=sector_roe.values,
        text=[f'{v:.1f}%' for v in sector_roe.values],
        textposition='outside',
        marker_color='#7b68ee',
        orientation='h',
        showlegend=False,
        hovertemplate='<b>%{y}</b><br>ROE: %{x:.1f}%<extra></extra>'
    ),
    row=1, col=1
)

median_roe = dataset['ROE_LTM'].median()
fig_setores.add_vline(x=median_roe, line_dash="dash", line_color="red", 
                      annotation_text="Mediana Geral", row=1, col=1)

fig_setores.add_trace(
    go.Bar(
        y=sector_debt.index,
        x=sector_debt.values,
        text=[f'{v:.1f}x' for v in sector_debt.values],
        textposition='outside',
        marker_color='#65c99f',
        orientation='h',
        showlegend=False,
        hovertemplate='<b>%{y}</b><br>Dívida/EBITDA: %{x:.1f}x<extra></extra>'
    ),
    row=1, col=2
)

median_debt = dataset['DEBT_TO_EBITDA_LTM'].median()
fig_setores.add_vline(x=median_debt, line_dash="dash", line_color="red",
                     annotation_text="Mediana Geral", row=1, col=2)

fig_setores.update_xaxes(title_text="ROE (%)", row=1, col=1)
fig_setores.update_xaxes(title_text="Dívida/EBITDA (x)", row=1, col=2)
fig_setores.update_yaxes(title_text="Setor")

fig_setores.update_layout(
    height=700,
    title_text='Análise Setorial: ROE vs Endividamento',
    title_x=0.5,
    title_font_size=16
)

fig_setores.show()

# Resumo Estatístico
print("\n📈 Resumo Estatístico:")

resumo = dataset.groupby('class_Composto').agg({
    'ENTITY_NAME': 'count',
    'ROE_LTM': 'median',
    'EV/EBITDA_LTM': 'median',
    'DEBT_TO_EBITDA_LTM': 'median',
    'NET_MARGIN_LTM': 'median',
    'composite_score': 'mean'
}).round(2)

resumo.columns = ['Qtd Empresas', 'ROE Mediano', 'EV/EBITDA Mediano', 
                  'Dívida/EBITDA Mediano', 'Margem Líquida', 'Score Médio']

display(resumo.reindex(ordem_classes))

print("Análise completa com Dividendos!")


🏆 Top 30 Empresas - Heatmap:


,ENTITY_NAME,TICKER,SECTOR,composite_score,class_Composto,class_CAGR,class_Margins,class_Multiplos,class_Qualidade,class_Profitability,class_Dividends
54,Baumer S.A. (BOVESPA:BALM4),BALM4,Health Care,11.000000,Excelente,Excelente,Excelente,Excelente,Excelente,Excelente,Boa
256,"Mills Locação, Serviços e Logística S.A. (BOVESPA:MILS3)",MILS3,Industrials,10.000000,Excelente,Excelente,Excelente,Ok,Excelente,Excelente,Excelente
138,Electro Aço Altona S.A. (BOVESPA:EALT4),EALT4,Materials,9.000000,Excelente,Excelente,Excelente,Excelente,Excelente,Excelente,Ruim
155,Eucatex S.A. Indústria e Comércio (BOVESPA:EUCA4),EUCA4,Materials,8.000000,Excelente,Boa,Boa,Excelente,Excelente,Ok,Excelente
399,Whirlpool S.A. (BOVESPA:WHRL4),WHRL4,Consumer Discretionary,8.000000,Excelente,Excelente,Ok,Ok,Excelente,Excelente,Excelente
244,Marcopolo S.A. (BOVESPA:POMO4),POMO4,Industrials,8.000000,Excelente,Excelente,Boa,Ruim,Excelente,Excelente,Excelente
266,MRS Logística S.A. (BOVESPA:MRSA3B),MRSA3B,Industrials,8.000000,Excelente,Boa,Excelente,Excelente,Boa,Boa,Boa
104,Companhia Energética de Minas Gerais - CEMIG (BOVESPA:CMIG4),CMIG4,Utilities,8.000000,Excelente,Boa,Ok,Excelente,Excelente,Boa,Excelente
210,ISA Energía Brasil S.A. (BOVESPA:ISAE4),ISAE4,Utilities,8.000000,Excelente,Boa,Excelente,Excelente,Boa,Ok,Excelente
75,Caixa Seguridade Participações S.A. (BOVESPA:CXSE3),CXSE3,Financials,7.000000,Boa,Excelente,Excelente,Péssima,Excelente,Excelente,Boa


C:\Users\Augusto\AppData\Local\Temp\ipykernel_23312\1267721564.py:59: FutureWarning:

DataFrame.applymap has been deprecated. Use DataFrame.map instead.




🏭 Análise Setorial:



📈 Resumo Estatístico:


,Qtd Empresas,ROE Mediano,EV/EBITDA Mediano,Dívida/EBITDA Mediano,Margem Líquida,Score Médio
class_Composto,,,,,,
Excelente,9,20.12,4.41,1.95,13.80,8.67
Boa,74,20.10,5.86,1.37,15.66,5.07
Ok,185,10.08,6.41,2.88,12.40,0.63
Ruim,127,4.35,6.51,3.70,-2.17,-4.69
Péssima,13,-19.19,13.24,6.37,-15.37,-8.54


Análise completa com Dividendos!


In [49]:
#análise de multiplos por setor
multiplos_ltm = ['EV/REVENUE_LTM', 'EV/EBITDA_LTM', 'EV/EBIT_LTM', 'EV/UFCF_LTM', 'PE_LTM', 'PBV_LTM']
multiplos_fwd = ['EV/REVENUE_FWD_2025', 'EV/EBITDA_FWD_2025', 'EV/EBIT_FWD_2025', 'PE_FWD_2025',
                 'EV/REVENUE_FWD_2026', 'EV/EBITDA_FWD_2026', 'EV/EBIT_FWD_2026', 'PE_FWD_2026',
                 'EV/REVENUE_FWD_2027', 'EV/EBITDA_FWD_2027', 'EV/EBIT_FWD_2027', 'PE_FWD_2027']

df_multiples = dataset.copy()

setor_stats = {}
for setor in df_multiples['SECTOR'].dropna().unique():
    setor_data = df_multiples[df_multiples['SECTOR'] == setor]
    setor_stats[setor] = {}
    for mult in multiplos_ltm + multiplos_fwd:
        if mult in df_multiples.columns:
            setor_stats[setor][mult] = setor_data[mult].median()

df_setor_multiples = pd.DataFrame(setor_stats).T
df_setor_multiples = df_setor_multiples.sort_values('EV/EBITDA_LTM', ascending=False)

print(f"Total de setores: {len(df_setor_multiples)}")


#gráfico 1: distribuição dos multiplos ltm por setor --------------------------------------------------------------------------------------------------------------------------------------

fig_ltm = make_subplots(
    rows=2, cols=3,
    subplot_titles=('EV/Revenue', 'EV/EBITDA', 'EV/EBIT', 'EV/UFCF', 'P/E', 'P/BV', 'Comparativo'),
    vertical_spacing=0.22, 
    horizontal_spacing=0.1
)

sector_thresholds = df_multiples['SECTOR'].value_counts().head(15).index
df_top = df_setor_multiples.loc[sector_thresholds]

# 1. EV/Revenue
fig_ltm.add_trace(
    go.Bar(
        x=df_top.index, 
        y=df_top['EV/REVENUE_LTM'],
        text=[f"{val:.1f}x" for val in df_top['EV/REVENUE_LTM']],
        textposition='outside',
        name='EV/Revenue', 
        marker_color='#7b68ee'
    ),
    row=1, col=1
)

# 2. EV/EBITDA
fig_ltm.add_trace(
    go.Bar(
        x=df_top.index, 
        y=df_top['EV/EBITDA_LTM'],
        text=[f"{val:.1f}x" for val in df_top['EV/EBITDA_LTM']],
        textposition='outside',
        name='EV/EBITDA', 
        marker_color="#505050"
    ),
    row=1, col=2
)

# 3. EV/EBIT
fig_ltm.add_trace(
    go.Bar(
        x=df_top.index, 
        y=df_top['EV/EBIT_LTM'],
        text=[f"{val:.1f}x" for val in df_top['EV/EBIT_LTM']],
        textposition='outside',
        name='EV/EBIT', 
        marker_color='#8a2be2'
    ),
    row=1, col=3
)

# 4 EV/UFCF
fig_ltm.add_trace(
    go.Bar(
        x=df_top.index, 
        y=df_top['EV/UFCF_LTM'],
        text=[f"{val:.1f}x" for val in df_top['EV/UFCF_LTM']],
        textposition='outside',
        name='EV/UFCF', 
        marker_color='#7b68ee'
    ),
    row=2, col=1
)

# 4. P/E
fig_ltm.add_trace(
    go.Bar(
        x=df_top.index, 
        y=df_top['PE_LTM'],
        text=[f"{val:.1f}x" for val in df_top['PE_LTM']],
        textposition='outside',
        name='P/E', 
        marker_color='#6a5acd'
    ),
    row=2, col=2
)

# 5. P/BV
fig_ltm.add_trace(
    go.Bar(
        x=df_top.index, 
        y=df_top['PBV_LTM'],
        text=[f"{val:.1f}x" for val in df_top['PBV_LTM']],
        textposition='outside',
        name='P/BV', 
        marker_color='#9932cc'
    ),
    row=2, col=3
)

fig_ltm.update_xaxes(tickangle=-45, tickfont=dict(size=9))
fig_ltm.update_layout(
    height=900, 
    showlegend=False,
    title_text="Múltiplos Medianos LTM por Setor",
    title_x=0.5,
    title_font_size=16
)

fig_ltm.show()

#gráfico 2: evolução dos multiplos forward por setor --------------------------------------------------------------------------------------------------------------------------------------------
print("\n=== Gráfico 2: Evolução dos Múltiplos Forward (2025-2027) ===")

fig_fwd = make_subplots(
    rows=2, cols=2,
    subplot_titles=('EV/Revenue Forward', 'EV/EBITDA Forward', 
                    'EV/EBIT Forward', 'P/E Forward'),
    vertical_spacing=0.20,  # Aumentado
    horizontal_spacing=0.12
)

top_10_sectors = df_multiples['SECTOR'].value_counts().head(10).index
df_top10 = df_setor_multiples.loc[top_10_sectors]

anos = ['2025', '2026', '2027']
cores = ["#5b1ba3", "#575757", "#31c4b0"]

# 1. EV/Revenue forward
for i, ano in enumerate(anos):
    col_name = f'EV/REVENUE_FWD_{ano}'
    if col_name in df_top10.columns:
        fig_fwd.add_trace(
            go.Bar(
                x=df_top10.index, 
                y=df_top10[col_name],
                text=[f"{val:.1f}x" for val in df_top10[col_name]],
                textposition='outside',
                textfont=dict(size=8),
                name=ano, 
                marker_color=cores[i]
            ),
            row=1, col=1
        )

# 2. EV/EBITDA forward
for i, ano in enumerate(anos):
    col_name = f'EV/EBITDA_FWD_{ano}'
    if col_name in df_top10.columns:
        fig_fwd.add_trace(
            go.Bar(
                x=df_top10.index, 
                y=df_top10[col_name],
                text=[f"{val:.1f}x" for val in df_top10[col_name]],
                textposition='outside',
                textfont=dict(size=8),
                name=ano, 
                marker_color=cores[i], 
                showlegend=False
            ),
            row=1, col=2
        )

# 3. EV/EBIT forward
for i, ano in enumerate(anos):
    col_name = f'EV/EBIT_FWD_{ano}'
    if col_name in df_top10.columns:
        fig_fwd.add_trace(
            go.Bar(
                x=df_top10.index, 
                y=df_top10[col_name],
                text=[f"{val:.1f}x" for val in df_top10[col_name]],
                textposition='outside',
                textfont=dict(size=8),
                name=ano, 
                marker_color=cores[i], 
                showlegend=False
            ),
            row=2, col=1
        )

# 4. P/E forward
for i, ano in enumerate(anos):
    col_name = f'PE_FWD_{ano}'
    if col_name in df_top10.columns:
        fig_fwd.add_trace(
            go.Bar(
                x=df_top10.index, 
                y=df_top10[col_name],
                text=[f"{val:.1f}x" for val in df_top10[col_name]],
                textposition='outside',
                textfont=dict(size=8),
                name=ano, 
                marker_color=cores[i], 
                showlegend=False
            ),
            row=2, col=2
        )

fig_fwd.update_xaxes(tickangle=-45, tickfont=dict(size=9))
fig_fwd.update_layout(
    height=900,  # Aumentado
    barmode='group',
    title_text="Evolução dos Múltiplos Forward (2025-2027)",
    title_x=0.5,
    title_font_size=16,
    legend=dict(
        orientation="h",
        yanchor="bottom",
        y=1.02,
        xanchor="right",
        x=1
    )
)

fig_fwd.show()


#gráfico 3: comparação ltm vs forward 2027
fig_compare = make_subplots(
    rows=1, cols=2,
    subplot_titles=('EV/EBITDA: LTM vs 2027', 'P/E: LTM vs 2027'),
    horizontal_spacing=0.15
)

df_scatter = df_setor_multiples.copy()
df_scatter['Setor'] = df_scatter.index

# EV/EBITDA comparison
fig_compare.add_trace(
    go.Scatter(
        x=df_scatter['EV/EBITDA_LTM'],
        y=df_scatter['EV/EBITDA_FWD_2027'],
        mode='markers+text',
        text=df_scatter['Setor'],
        textposition='top center',
        textfont=dict(size=10),
        marker=dict(size=10, color="#711c99", opacity=0.6),
        name='Setores'
    ),
    row=1, col=1
)

#linha de referência
max_ev = max(df_scatter['EV/EBITDA_LTM'].max(), df_scatter['EV/EBITDA_FWD_2027'].max())
fig_compare.add_trace(
    go.Scatter(x=[0, max_ev], y=[0, max_ev], 
               mode='lines', line=dict(dash='dash', color='gray'),
               showlegend=False),
    row=1, col=1
)

# P/E comparação
fig_compare.add_trace(
    go.Scatter(
        x=df_scatter['PE_LTM'],
        y=df_scatter['PE_FWD_2027'],
        mode='markers+text',
        text=df_scatter['Setor'],
        textposition='top center',
        textfont=dict(size=10),
        marker=dict(size=10, color="#0effcb", opacity=0.6),
        showlegend=False
    ),
    row=1, col=2
)

max_pe = max(df_scatter['PE_LTM'].max(), df_scatter['PE_FWD_2027'].max())
fig_compare.add_trace(
    go.Scatter(x=[0, max_pe], y=[0, max_pe], 
               mode='lines', line=dict(dash='dash', color='gray'),
               showlegend=False),
    row=1, col=2
)

fig_compare.update_xaxes(title_text="Múltiplo LTM", row=1, col=1)
fig_compare.update_xaxes(title_text="Múltiplo LTM", row=1, col=2)
fig_compare.update_yaxes(title_text="Múltiplo FWD 2027", row=1, col=1)
fig_compare.update_yaxes(title_text="Múltiplo FWD 2027", row=1, col=2)

fig_compare.update_layout(
    height=600,
    title_text="Comparação: Múltiplos LTM vs Forward 2027",
    title_x=0.5,
    title_font_size=16
)

fig_compare.show()


#heatmap dos multiplos ltm por setor
df_heatmap = df_setor_multiples[multiplos_ltm].copy()
df_heatmap = df_heatmap.loc[sector_thresholds]

fig_heatmap = go.Figure(data=go.Heatmap(
    z=df_heatmap.values,
    x=df_heatmap.columns,
    y=df_heatmap.index,
    colorscale=[[0, '#1a5c1a'], [0.2, '#28a745'], [0.4, '#90ee90'], 
                [0.5, '#ffffe0'], [0.6, '#ffd700'], [0.8, '#ff8c00'], [1, '#8b0000']],
    text=df_heatmap.round(1).values,
    texttemplate='%{text}x',
    textfont={"size": 10},
    colorbar=dict(title="Múltiplo", len=0.8)
))

fig_heatmap.update_layout(
    title='Heatmap: Múltiplos LTM por Setor',
    xaxis_title='Múltiplo',
    yaxis_title='Setor',
    height=700,
    width=1000,
    title_font_size=16
)

fig_heatmap.show()

df_heatmap_fwd = df_setor_multiples[multiplos_fwd].copy()
df_heatmap_fwd = df_heatmap_fwd.loc[sector_thresholds]
fig_heatmap_fwd = go.Figure(data=go.Heatmap(
    z=df_heatmap_fwd.values,
    x=df_heatmap_fwd.columns,
    y=df_heatmap_fwd.index,
    colorscale=[[0, '#1a5c1a'], [0.2, '#28a745'], [0.4, '#90ee90'], 
                [0.5, '#ffffe0'], [0.6, '#ffd700'], [0.8, '#ff8c00'], [1, '#8b0000']],
    text=df_heatmap_fwd.round(1).values,
    texttemplate='%{text}x',
    textfont={"size": 10},
    colorbar=dict(title="Múltiplo", len=0.8)
))

fig_heatmap_fwd.update_layout(
    title='Heatmap: Múltiplos Forward por Setor',
    xaxis_title='Múltiplo',
    yaxis_title='Setor',
    height=700,
    width=1000,
    title_font_size=16
)

fig_heatmap_fwd.show()

#tebela resumo
print("\n=== Tabela Resumo: Medianas por Setor ===")

colunas_resumo = ['EV/EBITDA_LTM', 'EV/EBITDA_FWD_2025', 'EV/EBITDA_FWD_2026', 'EV/EBITDA_FWD_2027', 
                  'PE_LTM', 'PE_FWD_2025', 'PE_FWD_2026', 'PE_FWD_2027',
                    'PBV_LTM']

df_resumo = df_setor_multiples[colunas_resumo].round(2)
df_resumo['Qtd_Empresas'] = df_multiples['SECTOR'].value_counts()
df_resumo = df_resumo.sort_values('Qtd_Empresas', ascending=False)

print("\nSetores por Número de Empresas:")
display(df_resumo.head(15).style.background_gradient(
    cmap='RdYlGn_r', 
    subset=colunas_resumo,
    vmin=0,
    vmax=25
).format(
    '{:.2f}x',
    subset=colunas_resumo
))


#análise de variação dos múltiplos
df_change = df_setor_multiples.copy()
df_change['EV_EBITDA_Change'] = ((df_change['EV/EBITDA_FWD_2027'] - df_change['EV/EBITDA_LTM']) / df_change['EV/EBITDA_LTM'] * 100)
df_change['PE_Change'] = ((df_change['PE_FWD_2027'] - df_change['PE_LTM']) / df_change['PE_LTM'] * 100)

print("\nSetores com maior variação negativa de múltiplos:")
compressao = df_change.nsmallest(5, 'EV_EBITDA_Change')[['EV/EBITDA_LTM', 'EV/EBITDA_FWD_2027', 'EV_EBITDA_Change']]
display(compressao)

print("\nSetores com maior expansão de múltiplos:")
expansao = df_change.nlargest(5, 'EV_EBITDA_Change')[['EV/EBITDA_LTM', 'EV/EBITDA_FWD_2027', 'EV_EBITDA_Change']]
display(expansao)

#análise das métricas de cash flow
df_fcf = dataset.copy()
df_fcf = df_fcf.dropna(subset=['EBITDA_MARGIN_LTM', 'LFCF_MARGIN_LTM', 'UFCF_MARGIN_LTM'])

sector_fcf = df_fcf.groupby('SECTOR').agg({
    'EBITDA_MARGIN_LTM': 'median',
    'LFCF_MARGIN_LTM': 'median',
    'UFCF_MARGIN_LTM': 'median',
    'ENTITY_NAME': 'count'
}).round(3)

sector_fcf = sector_fcf[sector_fcf['ENTITY_NAME'] >= 5]
sector_fcf = sector_fcf.sort_values('UFCF_MARGIN_LTM', ascending=False).head(12)

fig_fcf = go.Figure()

x_positions = list(range(len(sector_fcf)))
bar_width = 0.25

fig_fcf.add_trace(go.Bar(
    name='EBITDA Margin',
    x=[i - bar_width for i in x_positions],
    y=sector_fcf['EBITDA_MARGIN_LTM'],
    marker_color='#7b68ee',
    text=[f'{v:.1f}%' for v in sector_fcf['EBITDA_MARGIN_LTM']],
    textposition='outside',
    width=bar_width
))

fig_fcf.add_trace(go.Bar(
    name='LFCF Margin',
    x=x_positions,
    y=sector_fcf['LFCF_MARGIN_LTM'],
    marker_color='#65c99f',
    text=[f'{v:.1f}%' for v in sector_fcf['LFCF_MARGIN_LTM']],
    textposition='outside',
    width=bar_width
))

fig_fcf.add_trace(go.Bar(
    name='UFCF Margin',
    x=[i + bar_width for i in x_positions],
    y=sector_fcf['UFCF_MARGIN_LTM'],
    marker_color='#01abdf',
    text=[f'{v:.1f}%' for v in sector_fcf['UFCF_MARGIN_LTM']],
    textposition='outside',
    width=bar_width
))

fig_fcf.update_layout(
    title='Cash Conversion por Setor: EBITDA vs LFCF vs UFCF (Top 12)',
    xaxis=dict(
        tickvals=x_positions,
        ticktext=sector_fcf.index,
        tickangle=-45
    ),
    yaxis_title='Margin (%)',
    height=600,
    barmode='group',
    title_font_size=16,
    showlegend=True
)

fig_fcf.show()

Total de setores: 11



=== Gráfico 2: Evolução dos Múltiplos Forward (2025-2027) ===



=== Tabela Resumo: Medianas por Setor ===

Setores por Número de Empresas:


,EV/EBITDA_LTM,EV/EBITDA_FWD_2025,EV/EBITDA_FWD_2026,EV/EBITDA_FWD_2027,PE_LTM,PE_FWD_2025,PE_FWD_2026,PE_FWD_2027,PBV_LTM,Qtd_Empresas
Consumer Discretionary,5.94x,5.38x,4.78x,4.49x,6.86x,9.30x,8.05x,7.00x,0.96x,85
Industrials,6.70x,5.18x,4.64x,4.31x,9.70x,12.30x,7.10x,6.00x,1.43x,62
Real Estate,9.91x,8.39x,9.10x,8.67x,7.29x,5.50x,6.80x,5.40x,0.74x,49
Financials,6.41x,3.68x,6.31x,5.84x,8.08x,8.80x,7.50x,6.55x,1.30x,48
Materials,5.19x,5.71x,4.86x,4.22x,9.51x,8.95x,8.00x,6.70x,1.01x,42
Utilities,6.51x,7.98x,7.18x,6.68x,8.83x,12.90x,11.50x,10.20x,1.54x,38
Consumer Staples,6.31x,5.87x,5.67x,5.26x,12.81x,9.85x,9.75x,7.50x,1.17x,27
Communication Services,5.17x,4.94x,4.52x,4.26x,15.67x,12.50x,10.35x,10.50x,0.86x,18
Health Care,6.73x,6.60x,5.73x,5.30x,12.32x,12.50x,9.50x,7.35x,1.06x,16
Information Technology,10.40x,6.69x,5.54x,4.84x,25.80x,7.80x,6.35x,6.70x,0.54x,14



Setores com maior variação negativa de múltiplos:


,EV/EBITDA_LTM,EV/EBITDA_FWD_2027,EV_EBITDA_Change
Information Technology,10.405,4.840,-53.483902
Industrials,6.700,4.310,-35.671642
Consumer Discretionary,5.940,4.490,-24.410774
Health Care,6.730,5.295,-21.322437
Materials,5.190,4.220,-18.689788



Setores com maior expansão de múltiplos:


,EV/EBITDA_LTM,EV/EBITDA_FWD_2027,EV_EBITDA_Change
Utilities,6.51,6.675,2.534562
Financials,6.41,5.845,-8.814353
Real Estate,9.91,8.670,-12.512614
Consumer Staples,6.31,5.260,-16.640254
Communication Services,5.17,4.260,-17.601547
